In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,1.604227,...,75.411587,2.004234,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,2.224533,...,76.966756,1.726139,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,2.312330,...,78.493126,1.462602,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,2.332765,...,78.801800,1.217048,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,2.225842,...,79.547467,1.031926,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2002-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608320,...,78.229336,1.977754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,2002-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.453784,...,76.934612,2.093929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,2002-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071670,...,75.331548,2.191347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,2002-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.149319,...,74.405966,2.307342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.038360,-0.693716,...,0.029035,-0.182257,1.402525,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.221742,-0.603174,...,0.060353,0.138191,0.499197,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,1.400094,-0.351990,...,0.033064,0.452705,-0.356839,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,1.441606,0.189583,...,-0.109968,0.516308,-1.154465,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,1.224400,0.717316,...,-0.250986,0.669955,-1.755788,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.479209,0.697302,...,-0.498914,-0.113596,-0.437518,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.014731,0.264175,...,-0.419733,-0.176406,-0.385163,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.024464,0.049785,...,-0.321149,-0.283086,-0.415774,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.195334,-0.223773,...,-0.205138,-0.406765,-0.475393,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
259        NaN
260        NaN
261        NaN
262        NaN
263        NaN
Name: Bahia - Consumo de Cimento (t), Length: 264, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.038360,-0.693716,...,0.029035,-0.182257,1.402525,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.221742,-0.603174,...,0.060353,0.138191,0.499197,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,1.400094,-0.351990,...,0.033064,0.452705,-0.356839,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,1.441606,0.189583,...,-0.109968,0.516308,-1.154465,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,1.224400,0.717316,...,-0.250986,0.669955,-1.755788,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.097864,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,-0.463180,-0.124526,...,-0.089711,-0.552869,-0.500043,1.430739,0.617097,1.205171,-1.332406,1.004714,1.051323,1.273469
158,1.120412,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,-0.366119,0.218896,...,-0.138090,-0.420868,-0.369291,1.428086,0.590103,1.206522,-1.357999,1.007770,1.045502,1.268173
159,1.139591,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,-0.415933,0.632077,...,-0.258235,-0.273422,-0.389089,1.425433,0.561454,1.207873,-1.383592,1.010826,1.039682,1.262877
160,1.156903,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,-0.184106,1.099938,...,-0.393645,-0.149855,-0.325560,1.422780,0.523492,1.209225,-1.409185,1.013881,1.033862,1.257581


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
157    213.266
158    285.938
159    219.576
160    267.203
161    240.714
Name: Bahia - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
126,0.558660,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,0.398433,0.532679,...,-0.301039,0.974067,-0.100403,0.461215,1.205505,0.833268,0.194482,0.840990,1.014636,0.957509
127,0.586295,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,0.881864,0.430146,...,-0.305019,0.883592,0.125796,0.471044,1.203037,0.852943,0.147537,0.856945,1.027926,0.975019
128,0.614441,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,1.489586,0.052429,...,-0.212853,0.871901,0.231163,0.480874,1.178322,0.872618,0.100591,0.872900,1.041216,0.992528
129,0.639726,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,1.573881,-0.261929,...,-0.129927,0.796449,0.231164,0.490704,1.155459,0.892293,0.053646,0.888855,1.054506,1.010038
130,0.666428,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,1.385770,-0.590265,...,-0.036978,0.790608,0.097551,0.500533,1.124580,0.911968,0.006700,0.904809,1.067796,1.027547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.479209,0.697302,...,-0.498914,-0.113596,-0.437518,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.014731,0.264175,...,-0.419733,-0.176406,-0.385163,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.024464,0.049785,...,-0.321149,-0.283086,-0.415774,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.195334,-0.223773,...,-0.205138,-0.406765,-0.475393,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1204955363, 4288140589, 945062301, 1578163188, 674910152, 3928923232, 4023788313, 2178889877, 2797376664, 2469496785, 1623094374, 139938307, 4197823081, 316891619, 505937130, 4198836269, 3122020789, 3420567170, 1474563229, 3714789750, 2418111916, 4170547530, 945979248, 2394054202, 326598428, 2600141495, 3696591145, 1566232679, 2536234443, 3335621420, 3550578555, 1410184816, 3896969222, 1627487441, 2914332535, 551149131, 3554704798, 4192696033, 2391300185, 3816837603, 1505854832, 130677964, 156634971, 1611721355, 3441167327, 1580430504, 12116113, 903321005, 1202636046, 355629169]


Step: 0 ___________________________________________
val_loss: 553.7412109375
winner_seed: 1204955363


Step: 1 ___________________________________________
val_loss: 613.1414184570312


Step: 2 ___________________________________________
val_loss: 570.1702270507812


Step: 3 ___________________________________________
val_loss: 1181.860107421875


Step: 4 ___________________________________________
val_loss: 

2023-10-20 11:45:04.327227: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 589.1661987304688


Step: 10 ___________________________________________
val_loss: 512.1698608398438
winner_seed: 1623094374


Step: 11 ___________________________________________
val_loss: 878.75244140625


Step: 12 ___________________________________________
val_loss: 622.929443359375


Step: 13 ___________________________________________
val_loss: 783.0830078125


Step: 14 ___________________________________________
val_loss: 637.3547973632812


Step: 15 ___________________________________________
val_loss: 579.6981811523438


Step: 16 ___________________________________________


2023-10-20 11:49:51.725653: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 549.134521484375


Step: 17 ___________________________________________
val_loss: 607.1585083007812


Step: 18 ___________________________________________
val_loss: 903.810546875


Step: 19 ___________________________________________
val_loss: 639.9957275390625


Step: 20 ___________________________________________
val_loss: 507.9351806640625
winner_seed: 2418111916


Step: 21 ___________________________________________
val_loss: 568.7835693359375


Step: 22 ___________________________________________
val_loss: 544.49462890625


Step: 23 ___________________________________________
val_loss: 561.6903686523438


Step: 24 ___________________________________________
val_loss: 621.6878051757812


Step: 25 ___________________________________________
val_loss: 537.5923461914062


Step: 26 ___________________________________________
val_loss: 668.2783203125


Step: 27 ___________________________________________
val_loss: 672.629638671875


Step: 28 ___________________________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 120ms/step - loss: 57661.2539 - val_loss: 3894.6025
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 2974.9497 - val_loss: 2896.0864
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 2919.0327 - val_loss: 4516.9863
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 4062.4526 - val_loss: 4475.5781
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 3164.9417 - val_loss: 3650.8035
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 3796.5295 - val_loss: 2975.1572
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 3192.3621 - val_loss: 2777.5771
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 2795.3298 - val_loss: 2661.8411
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 3203.4111 - val_loss: 2825.4104
Epoch 10/10000
4/4 [========================

4/4 [==============================] - 0s 11ms/step - loss: 328.9410 - val_loss: 703.7424
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 301.8081 - val_loss: 592.1796
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 299.7668 - val_loss: 653.8931
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 273.0102 - val_loss: 634.6710
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 268.2903 - val_loss: 611.0404
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 288.1713 - val_loss: 592.8536
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 280.8788 - val_loss: 646.5765
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 237.6575 - val_loss: 630.5984
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 255.4982 - val_loss: 1033.6656
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - l

4/4 [==============================] - 0s 11ms/step - loss: 214.7852 - val_loss: 639.7597
Epoch 156/10000
4/4 [==============================] - 0s 11ms/step - loss: 209.4821 - val_loss: 670.7412
Epoch 157/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.2251 - val_loss: 657.3181
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 216.0056 - val_loss: 748.8733
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 200.8542 - val_loss: 730.6512
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 202.2718 - val_loss: 753.0575
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 243.6872 - val_loss: 766.3827
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 235.8396 - val_loss: 702.1755
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 209.4466 - val_loss: 613.5001
Epoch 164/10000
4/4 [==============================] - 0s 11ms/

4/4 [==============================] - 0s 9ms/step - loss: 222.4802 - val_loss: 612.8777
Epoch 233/10000
4/4 [==============================] - 0s 10ms/step - loss: 216.3228 - val_loss: 603.0468
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 198.7170 - val_loss: 599.7838
Epoch 235/10000
4/4 [==============================] - 0s 11ms/step - loss: 198.8653 - val_loss: 722.8079
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 275.1929 - val_loss: 625.3996
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 214.3308 - val_loss: 611.9238
Epoch 238/10000
4/4 [==============================] - 0s 11ms/step - loss: 182.1263 - val_loss: 615.1033
Epoch 239/10000
4/4 [==============================] - 0s 12ms/step - loss: 225.9146 - val_loss: 555.5974
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 189.3769 - val_loss: 554.1085
Epoch 241/10000
4/4 [==============================] - 0s 11ms/

4/4 [==============================] - 0s 10ms/step - loss: 162.1352 - val_loss: 688.9948
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 155.4392 - val_loss: 681.6227
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 177.8956 - val_loss: 662.1125
Epoch 312/10000
4/4 [==============================] - 0s 12ms/step - loss: 168.6901 - val_loss: 638.1435
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.4355 - val_loss: 646.7757
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 171.9503 - val_loss: 630.0779
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 168.4564 - val_loss: 647.4995
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 156.6684 - val_loss: 675.1277
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 162.0480 - val_loss: 684.4794
Epoch 318/10000
4/4 [==============================] - 0s 11ms

...

4/4 [==============================] - 0s 10ms/step - loss: 124.9100 - val_loss: 659.5002
Epoch 772/10000
4/4 [==============================] - 0s 11ms/step - loss: 110.5276 - val_loss: 690.3003
Epoch 773/10000
4/4 [==============================] - 0s 10ms/step - loss: 108.9885 - val_loss: 667.9261
Epoch 774/10000
4/4 [==============================] - 0s 9ms/step - loss: 163.6107 - val_loss: 654.2170
Epoch 775/10000
4/4 [==============================] - 0s 10ms/step - loss: 155.2506 - val_loss: 651.3510
Epoch 776/10000
4/4 [==============================] - 0s 11ms/step - loss: 122.1701 - val_loss: 651.5846
Epoch 777/10000
4/4 [==============================] - 0s 10ms/step - loss: 146.4843 - val_loss: 689.1733
Epoch 778/10000
4/4 [==============================] - 0s 9ms/step - loss: 116.5097 - val_loss: 664.5082
Epoch 779/10000
4/4 [==============================] - 0s 11ms/step - loss: 114.6571 - val_loss: 662.8682
Epoch 780/10000
4/4 [==============================] - 0s 9ms/st

4/4 [==============================] - 0s 10ms/step - loss: 116.3985 - val_loss: 627.8659
Epoch 849/10000
4/4 [==============================] - 0s 10ms/step - loss: 127.2877 - val_loss: 669.7250
Epoch 850/10000
4/4 [==============================] - 0s 9ms/step - loss: 120.0099 - val_loss: 700.2278
Epoch 851/10000
4/4 [==============================] - 0s 9ms/step - loss: 125.1767 - val_loss: 710.7169
Epoch 852/10000
4/4 [==============================] - 0s 10ms/step - loss: 100.3157 - val_loss: 696.2103
Epoch 853/10000
4/4 [==============================] - 0s 9ms/step - loss: 125.8556 - val_loss: 699.8322
Epoch 854/10000
4/4 [==============================] - 0s 9ms/step - loss: 116.9710 - val_loss: 685.5668
Epoch 855/10000
4/4 [==============================] - 0s 11ms/step - loss: 145.5002 - val_loss: 673.6547
Epoch 856/10000
4/4 [==============================] - 0s 11ms/step - loss: 124.0732 - val_loss: 711.1097
Epoch 857/10000
4/4 [==============================] - 0s 11ms/ste

4/4 [==============================] - 0s 11ms/step - loss: 89.9114 - val_loss: 728.7313
Epoch 926/10000
4/4 [==============================] - 0s 11ms/step - loss: 100.9656 - val_loss: 692.5994
Epoch 927/10000
4/4 [==============================] - 0s 9ms/step - loss: 98.0062 - val_loss: 698.6209
Epoch 928/10000
4/4 [==============================] - 0s 10ms/step - loss: 87.9872 - val_loss: 680.9542
Epoch 929/10000
4/4 [==============================] - 0s 10ms/step - loss: 115.3391 - val_loss: 689.3710
Epoch 930/10000
4/4 [==============================] - 0s 11ms/step - loss: 104.6694 - val_loss: 687.0225
Epoch 931/10000
4/4 [==============================] - 0s 10ms/step - loss: 109.9089 - val_loss: 689.6311
Epoch 932/10000
4/4 [==============================] - 0s 13ms/step - loss: 110.4585 - val_loss: 698.7850
Epoch 933/10000
4/4 [==============================] - 0s 12ms/step - loss: 103.4998 - val_loss: 665.5487
Epoch 934/10000
4/4 [==============================] - 0s 11ms/ste

Epoch 1003/10000
4/4 [==============================] - 0s 11ms/step - loss: 107.2689 - val_loss: 745.6584
Epoch 1004/10000
4/4 [==============================] - 0s 10ms/step - loss: 98.5698 - val_loss: 787.7244
Epoch 1005/10000
4/4 [==============================] - 0s 9ms/step - loss: 104.5764 - val_loss: 726.3801
Epoch 1006/10000
4/4 [==============================] - 0s 11ms/step - loss: 117.2129 - val_loss: 748.4990
Epoch 1007/10000
4/4 [==============================] - 0s 10ms/step - loss: 98.4683 - val_loss: 733.4036
Epoch 1008/10000
4/4 [==============================] - 0s 13ms/step - loss: 134.1000 - val_loss: 776.1608
Epoch 1009/10000
4/4 [==============================] - 0s 11ms/step - loss: 95.5503 - val_loss: 767.2470
Epoch 1010/10000
4/4 [==============================] - 0s 11ms/step - loss: 108.7551 - val_loss: 758.0757
Epoch 1011/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.6812 - val_loss: 765.0439
Epoch 1012/10000
4/4 [====================

4/4 [==============================] - 0s 10ms/step - loss: 94.3851 - val_loss: 823.7172
Epoch 1081/10000
4/4 [==============================] - 0s 11ms/step - loss: 91.3278 - val_loss: 804.6003
Epoch 1082/10000
4/4 [==============================] - 0s 11ms/step - loss: 91.5503 - val_loss: 778.2991
Epoch 1083/10000
4/4 [==============================] - 0s 10ms/step - loss: 104.5805 - val_loss: 700.6866
Epoch 1084/10000
4/4 [==============================] - 0s 14ms/step - loss: 67.2025 - val_loss: 777.7902
Epoch 1084: early stopping


In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,269.392365,268.480896,266.625122,266.005981,263.088898,256.995148,244.792038,245.031662,253.702759,258.802612,263.399261,266.450043,267.620056,264.992401,262.941376,256.804626,250.55426,243.252045,242.999542,243.297684,244.29599,250.122833,248.844467,248.875122,249.001831,249.501114,247.965118,244.582306,243.515015,243.249268
Target,250.101,277.528,270.092,278.146,257.458,231.748,268.336,223.453,241.464,238.901,191.989,272.452,261.009,292.688,258.881,276.879,255.774,208.326,291.428,249.43,241.612,252.303,281.912,200.213,270.511,281.466,285.535,328.259,298.078,264.838
Error,19.291367,9.047119,3.466888,12.140015,5.63089,25.247147,23.543961,21.578659,12.238754,19.901611,71.410263,6.001953,6.611053,27.695587,4.060364,20.074371,5.219742,34.926041,48.428467,6.132309,2.68399,2.180161,33.06752,48.662125,21.509155,31.96489,37.569885,83.676697,54.562988,21.588745


In [38]:
display(mae)
display(mape)

24.003757

0.09401667

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined